In [2]:
import numpy as np
import pandas as pd

# Data Description

In [5]:
# Shared drive read_csv
wids = pd.read_csv('../data/training_v2.csv')
print("This data has {} rows and {} columns.".format(wids.shape[0], wids.shape[1]))

This data has 91713 rows and 186 columns.


In [6]:
wids[['encounter_id', 'patient_id', 'hospital_id']].nunique()
print("Patient_id and encounter_id are unique.")

Patient_id and encounter_id are unique.


In [7]:
wids['hospital_death'].value_counts(normalize=True)
print("We have an imbalanced dataset. Only around 8% of our patients have hospital_death =1.")

We have an imbalanced dataset. Only around 8% of our patients have hospital_death =1.


# Feature Engineering
We are going to create features relevant to a patient's possibility of death. The following are the functions for feature engineering. 

In [8]:
# Viviana
def gcs_finalscore(df):
    """
    Parameters
    ----------
    df : dataframe
        full dataframe.

    Returns
    -------
    df['gcs_score'] : series
        Gasglow Coma Scale Score.
    """
    df['gcs_score'] =  df['gcs_eyes_apache'] + df['gcs_verbal_apache'] + df['gcs_motor_apache']
    return df['gcs_score']

def creatine_risks(df):
    """

    Parameters
    ----------
    df : dataframe
        full dataframe.

    Returns
    -------
    df['creatine_risk'] : series
        Series defines risk from creatine (0) if they are below the risk threshold and (1) if they are at risk .

    """
    
    df['creatine_risk'] = pd.cut(df['creatinine_apache'],bins = [0,1.1,np.inf], labels = [0,1])
    
    return df['creatine_risk']
  

def bilirubin_risk (df):
    """ Column that tells us if the individual is at risk liver problems. High bilirubin is associated with a higher risk.

    Parameters
    ----------
    df : Dataframe
        original dataframe.

    Returns
    -------
    df['bilirubin_risk']: Series
        a colum of 0s and 1s, 1 if you are above .6, 0 otherwise.

    """
    df['bilirubin_risk'] = pd.cut(df['bilirubin_apache'], bins = [0,.6,np.inf], labels = [0,1])

    return df['bilirubin_risk']

def glucose_risk (df):
    """ Column that tells us if the individual is at risk. High glucose above 240 is associated with a higher risk of ICU death. 

    Parameters
    ----------
    df : Dataframe
        original dataframe.

    Returns
    -------
    df['glucose_risk']: Series
        a colum of 0s and 1s, 1 if you are above 240, 0 otherwise.

    """
    df['glucose_risk'] = pd.cut(df['glucose_apache'], bins = [0,240,np.inf], labels = [0,1])


    return df['glucose_risk']

In [9]:
wids['gcs_score'] = gcs_finalscore(wids)
wids['creatine_risk'] = creatine_risks(wids)
wids['bilirubin_risk'] = bilirubin_risk(wids)
wids['glucose_risk'] = glucose_risk(wids)

In [10]:
# Jessica

#number of test done in an hour
def h1_test(df):
    column = df.columns
    h1_list = []
    for i in column:
        if (i[:2]=='h1'):
            h1_list.append(i)
    return df[h1_list].notnull().sum(axis=1)/2


#number of test done in a day
def d1_test(df):
    column = df.columns
    d1_list = []
    for i in column:
        if (i[:2]=='d1'):
            d1_list.append(i)
    return df[d1_list].notnull().sum(axis=1)/2



#number of test are missing
def missing_function(df):
    df_sub = df.loc[:,'d1_diasbp_invasive_max':'h1_pao2fio2ratio_min']
    return df_sub.isnull().sum(axis=1)/2


#number of chronic conditions
def chronic_function(df):
    df_sub = df.loc[:, 'aids':'solid_tumor_with_metastasis'] 
    return df_sub.sum(axis=1)

In [11]:
wids['h1_test_total'] = h1_test(wids)
wids['d1_test_total'] = d1_test(wids)
wids['missing_test_total'] = missing_function(wids)
wids['cc_total'] = chronic_function(wids)

In [12]:
# Vamika

def avg_h1_cols(df):
    '''
    Creates column with average h1 data using col_max and col_min variables

    Parameters:
    -----------------------------
    df                pd.DataFrame

    Return:
    -----------------------------
    avg_h1_data         pd.DataFrame

    '''
    avg_h1_list = ['h1_diasbp', 'h1_heartrate', 'h1_mbp', 'h1_resprate', 'h1_spo2', 'h1_sysbp',
                        'h1_temp', 'h1_albumin', 'h1_bilirubin', 'h1_bun', 'h1_calcium', 'h1_creatinine',
                        'h1_glucose', 'h1_hco3', 'h1_hemaglobin', 'h1_hematocrit', 'h1_inr', 'h1_lactate', 
                        'h1_platelets', 'h1_potassium', 'h1_sodium', 'h1_wbc', 'h1_arterial_pco2', 
                        'h1_arterial_ph', 'h1_arterial_po2', 'h1_pao2fio2ratio']
    avg_h1_data = pd.DataFrame()
    for col in avg_h1_list:
        avg_h1_data[col + '_avg'] = df[[col + '_min', col + '_max']].mean(axis=1)
    return(avg_h1_data) 

    

def diff_h1d1_cols(df):
    '''
    Creates column with difference in h1 and d1 data using col_max and col_min variables

    Parameters:
    -----------------------------
    df                pd.DataFrame

    Return:
    -----------------------------
    diff_h1d1_data         pd.DataFrame

    '''
    diff_h1d1_list = ['h1_diasbp', 'h1_heartrate', 'h1_mbp', 'h1_resprate', 'h1_spo2', 'h1_sysbp',
                        'h1_temp', 'h1_albumin', 'h1_bilirubin', 'h1_bun', 'h1_calcium', 'h1_creatinine',
                        'h1_glucose', 'h1_hco3', 'h1_hemaglobin', 'h1_hematocrit', 'h1_inr', 'h1_lactate', 
                        'h1_platelets', 'h1_potassium', 'h1_sodium', 'h1_wbc', 'h1_arterial_pco2', 
                        'h1_arterial_ph', 'h1_arterial_po2', 'h1_pao2fio2ratio', 'd1_diasbp','d1_heartrate',
                        'd1_mbp','d1_resprate','d1_spo2', 'd1_sysbp', 'd1_temp', 'd1_albumin',
                        'd1_bilirubin', 'd1_bun', 'd1_calcium', 'd1_creatinine', 'd1_glucose', 'd1_hco3',
                        'd1_hemaglobin', 'd1_hematocrit', 'd1_inr', 'd1_lactate', 'd1_platelets', 'd1_potassium',
                        'd1_sodium', 'd1_wbc', 'd1_arterial_pco2', 'd1_arterial_ph', 'd1_arterial_po2', 'd1_pao2fio2ratio']
    diff_h1d1_data = pd.DataFrame()
    for col in diff_h1d1_list:
        diff_h1d1_data[col + '_diff'] = df[col + '_max'] - df[col + '_min']
    return(diff_h1d1_data)

    
def med_diff_cols(df):
    '''
    Creates column with difference in median(h1) and median(d1) data using col_max and col_min variables

    Parameters:
    -----------------------------
    df                pd.DataFrame

    Return:
    -----------------------------
    med_diff_data         pd.DataFrame

    '''

    h1d1_list = ['diasbp', 'heartrate', 'mbp', 'resprate', 'spo2', 'sysbp', 'temp', 'albumin', 'bilirubin', 'bun',
                 'calcium', 'creatinine', 'glucose', 'hco3', 'hemaglobin', 'hematocrit', 'inr', 'lactate', 'platelets',
                 'potassium', 'sodium', 'wbc', 'arterial_pco2', 'arterial_ph', 'arterial_po2', 'pao2fio2ratio']
    med_diff_data = pd.DataFrame()
    for col in h1d1_list:
        med_diff_data[col + 'med_diff'] = ((df[['d1_' + col + '_min', 'd1_' + col + '_max']].median(axis=1)) - df[['h1_' + col + '_min', 'h1_' + col + '_max']].median(axis=1))
    return(med_diff_data)

def h1_ppv(df):
    '''
    Creates column with the h1 blood pressure ratio (sys / dias)

    Parameters:
    -----------------------------
    df                pd.DataFrame

    Return:
    -----------------------------
    h1_ppv         pd.DataFrame

    '''
    h1_ppv = pd.DataFrame()
    h1_ppv['h1_ppv'] = (df[['h1_sysbp_max', 'h1_sysbp_min']].median(axis=1)) / (df[['h1_diasbp_max', 'h1_diasbp_min']].median(axis=1))
    return(h1_ppv)


def d1_ppv(df):
    '''
    Creates column with the d1 blood pressure ratio (sys / dias)

    Parameters:
    -----------------------------
    df                pd.DataFrame

    Return:
    -----------------------------
    d1_ppv         pd.DataFrame

    '''
    d1_ppv = pd.DataFrame()
    d1_ppv['d1_ppv'] = (df[['d1_sysbp_max', 'd1_sysbp_min']].median(axis=1)) / (df[['d1_diasbp_max', 'd1_diasbp_min']].median(axis=1))
    return(d1_ppv)

In [13]:
wids = pd.concat([wids, avg_h1_cols(wids)], axis=1)

In [14]:
wids = pd.concat([wids, diff_h1d1_cols(wids), med_diff_cols(wids), h1_ppv(wids), d1_ppv(wids)], axis=1)

In [15]:
# Meghna

# Flag for Urine output level
def urine_flag(df):
    '''
    Creates 1 for patients with less than 800 mL of urine output in the first 24 hours
    0 Otherwise

    Parameters:
    -----------------------------
    df                pd.DataFrame

    Return:
    -----------------------------
    low_urine         pd.Series

    '''
    bins = [0, 800, np.inf]
    names = [1, 0]
    
    low_urine = pd.cut(df['urineoutput_apache'], bins, labels=names, include_lowest = True)
    return pd.Series(low_urine)

# Flag for white blood cells count
def wbc_level(df):
    '''
    Creates three levels for different levels of White Blood Cells
    Level = 0 --> wbc <10
    Level = 1 --> 10 <= wbc <s10
    Level = 2 --> 30 <= wbc

    Parameters:
    -----------------------------
    df                pd.DataFrame

    Return:
    -----------------------------
    wbc_level         pd.Series

    '''
    bins = [0, 10, 30, np.inf]
    names = [0, 1, 2]
    
    wbc_level = pd.cut(df['wbc_apache'], bins, labels=names, include_lowest = True)
    return pd.Series(wbc_level)

# Flag for lactate level
def lactate_level(df):
    '''
    Creates 1 for patients with lactate greater than 2.2 in the first 24 hours
    0 Otherwise
    
    Used d1_lactate_max as a more conservative measure

    Parameters:
    -----------------------------
    df                pd.DataFrame

    Return:
    -----------------------------
    high_lactate       pd.Series

    '''
    bins = [0, 2.2, np.inf]
    names = [1, 0]
    
    high_lactate = pd.cut(df['d1_lactate_max'], bins, labels=names, include_lowest = True)
    return pd.Series(high_lactate)

# Flag for platelets level
def platelets_level(df):
    '''
    Creates 1 for patients with platelets less than 100 in the first 24 hours
    0 Otherwise
    
    Used d1_plateslets_min as a more conservative measure

    Parameters:
    -----------------------------
    df                pd.DataFrame

    Return:
    -----------------------------
    low_platelets    pd.Series

    '''
    bins = [0, 100, np.inf]
    names = [1, 0]
    
    low_platelets = pd.cut(df['d1_platelets_min'], bins, labels=names, include_lowest = True)
    return pd.Series(low_platelets)



In [16]:
wids['urine_flag'] = urine_flag(wids)
wids['wbc_level'] = wbc_level(wids)
wids['lactate_level'] = lactate_level(wids)
wids['platelets_level'] = platelets_level(wids)

In [17]:
## Jenny
# Diff between first_hour and first_day max/min variables
def diff_h1_d1(df):
    """
    Diff between first_hour and first_day max/min variables. 
    It shows the drop/increase of the measures (max/min).
    
    Parameters:
    -----------------------
    df:                pd.DataFrame
    
    
    Return:
    -----------------------
    output:            pd.DataFrame or pd.Series
                       one or multiple columns that went through the feature engineering. 
    """
    
    
    hd = df[['d1_diasbp_invasive_max', 'd1_diasbp_invasive_min',
       'd1_diasbp_max', 'd1_diasbp_min', 'd1_diasbp_noninvasive_max',
       'd1_diasbp_noninvasive_min', 'd1_heartrate_max',
       'd1_heartrate_min', 'd1_mbp_invasive_max', 'd1_mbp_invasive_min',
       'd1_mbp_max', 'd1_mbp_min', 'd1_mbp_noninvasive_max',
       'd1_mbp_noninvasive_min', 'd1_resprate_max', 'd1_resprate_min',
       'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_invasive_max',
       'd1_sysbp_invasive_min', 'd1_sysbp_max', 'd1_sysbp_min',
       'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min',
       'd1_temp_max', 'd1_temp_min', 'h1_diasbp_invasive_max',
       'h1_diasbp_invasive_min', 'h1_diasbp_max', 'h1_diasbp_min',
       'h1_diasbp_noninvasive_max', 'h1_diasbp_noninvasive_min',
       'h1_heartrate_max', 'h1_heartrate_min', 'h1_mbp_invasive_max',
       'h1_mbp_invasive_min', 'h1_mbp_max', 'h1_mbp_min',
       'h1_mbp_noninvasive_max', 'h1_mbp_noninvasive_min',
       'h1_resprate_max', 'h1_resprate_min', 'h1_spo2_max', 'h1_spo2_min',
       'h1_sysbp_invasive_max', 'h1_sysbp_invasive_min', 'h1_sysbp_max',
       'h1_sysbp_min', 'h1_sysbp_noninvasive_max',
       'h1_sysbp_noninvasive_min', 'h1_temp_max', 'h1_temp_min',
       'd1_albumin_max', 'd1_albumin_min', 'd1_bilirubin_max',
       'd1_bilirubin_min', 'd1_bun_max', 'd1_bun_min', 'd1_calcium_max',
       'd1_calcium_min', 'd1_creatinine_max', 'd1_creatinine_min',
       'd1_glucose_max', 'd1_glucose_min', 'd1_hco3_max', 'd1_hco3_min',
       'd1_hemaglobin_max', 'd1_hemaglobin_min', 'd1_hematocrit_max',
       'd1_hematocrit_min', 'd1_inr_max', 'd1_inr_min', 'd1_lactate_max',
       'd1_lactate_min', 'd1_platelets_max', 'd1_platelets_min',
       'd1_potassium_max', 'd1_potassium_min', 'd1_sodium_max',
       'd1_sodium_min', 'd1_wbc_max', 'd1_wbc_min', 'h1_albumin_max',
       'h1_albumin_min', 'h1_bilirubin_max', 'h1_bilirubin_min',
       'h1_bun_max', 'h1_bun_min', 'h1_calcium_max', 'h1_calcium_min',
       'h1_creatinine_max', 'h1_creatinine_min', 'h1_glucose_max',
       'h1_glucose_min', 'h1_hco3_max', 'h1_hco3_min',
       'h1_hemaglobin_max', 'h1_hemaglobin_min', 'h1_hematocrit_max',
       'h1_hematocrit_min', 'h1_inr_max', 'h1_inr_min', 'h1_lactate_max',
       'h1_lactate_min', 'h1_platelets_max', 'h1_platelets_min',
       'h1_potassium_max', 'h1_potassium_min', 'h1_sodium_max',
       'h1_sodium_min', 'h1_wbc_max', 'h1_wbc_min',
       'd1_arterial_pco2_max', 'd1_arterial_pco2_min',
       'd1_arterial_ph_max', 'd1_arterial_ph_min', 'd1_arterial_po2_max',
       'd1_arterial_po2_min', 'd1_pao2fio2ratio_max',
       'd1_pao2fio2ratio_min', 'h1_arterial_pco2_max',
       'h1_arterial_pco2_min', 'h1_arterial_ph_max', 'h1_arterial_ph_min',
       'h1_arterial_po2_max', 'h1_arterial_po2_min',
       'h1_pao2fio2ratio_max', 'h1_pao2fio2ratio_min']]
    
    # Keep the synthesized ones. 
    hd = hd[hd.columns.drop(hd.filter(regex = 'invasive'))]

    hd = hd.reindex(sorted(hd.columns), axis=1)
    
    # h1-d1
    cols = hd.iloc[:, 52:].columns
    cols = [i.strip('h1_') + '_change' for i in cols] 

    output = pd.DataFrame(hd.iloc[:, 52:].values - hd.iloc[:, :52].values, columns=cols)
    
    return output



# Apache diagnosis - The part before decimal
def grab_main_cat(col):
    """
    Return the main category for apache diagnosis. (apache_3j_diagnosis and apache_2_diagnosis)

    Parameters
    -------------------
    col                 pd.Series
                        The variables we want to extract the main category.

    Return
    -------------------
    output              pd.Series
                        Main categories of diagnosis. 

    """
    output = col.astype('str').str.split('.', n=1, expand=True)[0]
    output = output.astype(str)
    
    return output


In [18]:
wids = pd.concat([wids, diff_h1_d1(wids)], axis=1)

In [19]:
wids['apache_3j_diagnosis'] = grab_main_cat(wids['apache_3j_diagnosis'])
wids['apache_2_diagnosis'] = grab_main_cat(wids['apache_2_diagnosis'])

In [20]:
#wids.to_csv('wids_featured', index=False)

In [26]:
wids_featured = wids.copy()

In [34]:
wids_featured.shape

(91713, 356)

In [57]:
wids_featured['apache_2_diagnosis'] = wids_featured['apache_2_diagnosis'].astype(str)
wids_featured['apache_3j_diagnosis'] = wids_featured['apache_2_diagnosis'].astype(str)

# Data Processing 

In [27]:
# null values 
def null_values(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [28]:
categorical_cols = ['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 
                    'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 
                    'apache_2_bodysystem', 'apache_2_diagnosis', 'apache_3j_diagnosis']


In [48]:
bool_cols = [col for col in wids_featured if np.isin(wids_featured[col].dropna().unique(), [0, 1]).all()]
binary_cols = [i for i in bool_cols if i not in ['solid_tumor_with_metastasis', 'inrmed_diff', 'inr_max_change', 'inr_min_change','hospital_death']]
binary_cols

['elective_surgery',
 'readmission_status',
 'apache_post_operative',
 'arf_apache',
 'gcs_unable_apache',
 'intubated_apache',
 'ventilated_apache',
 'aids',
 'cirrhosis',
 'diabetes_mellitus',
 'hepatic_failure',
 'immunosuppression',
 'leukemia',
 'lymphoma',
 'creatine_risk',
 'bilirubin_risk',
 'glucose_risk',
 'urine_flag',
 'lactate_level',
 'platelets_level']

In [53]:
target_col = ['hospital_death']

In [54]:
numerical_cols = list(set(wids_featured.columns) - set((categorical_cols + binary_cols + target_col)))

In [56]:
len(categorical_cols) + len(binary_cols) + len(numerical_cols)  + len(target_col)

356

In [ ]:
# Dropping columns that have over than 70% missing values. 
missing_raw = pd.DataFrame(wids.isna().sum()/wids.shape[0]).reset_index()

cols_drop = missing_raw.loc[missing_raw[0]>0.7, 'index']

wids_featured.drop(cols_drop, axis=1, inplace=True)

In [ ]:
def datacleansing_analysisready(df, target_col, cat_cols, num_cols, binary_cols, cols_drop):
    """ 

    Parameters
    ----------
    df : type
        Description of parameter `df`.
    target_col : type
        Description of parameter `target_col`.
    cat_cols : type
        Description of parameter `cat_cols`.
    num_cols : type
        Description of parameter `num_cols`.
    binary_cols : type
        Description of parameter `binary_cols`.
    cols_drop : type
        Description of parameter `cols_drop`.

    Returns
    -------
    type
        Description of returned object.

    """
    # drop the null values from the dataset
    df.drop(cols_drop, axis=1, inplace=True)


    # make a copy of the df dataframe
    X_df = df.copy()

    # imputing less than 10 % with general median
    wids_num = X_df.drop(cat+bool_cols, axis=1)
    wids_num_missing = pd.DataFrame(wids_num.isna().sum()/wids_num.shape[0]).reset_index()
    wids_num10 = wids_num_missing[wids_num_missing[0] < .1]['index'].to_list()

    # assigned the imputation to the columns
    X_df[wids_num_missing.loc[wids_num_missing[0]<0.1, 'index']] = X_df[wids_num_missing.loc[wids_num_missing[0]<0.1, 'index']].apply(lambda x: x.fillna(x.median()))


    # impute the numerical cols that were not less than 10
    numerical_cols = list(set(numerical_cols) - set(wids_num10))

    # creating the bins for apache_4a_icu_death_prob to use for numerical cols imputation
    X_df['tmp'] = pd.cut(X_df['apache_4a_icu_death_prob'],9)
    X_df['tmp'] = X_df['tmp'].astype(str)

    # go through numerical cols and impute them with the apache_4a_icu_death_prob group mean
    for i in numerical_cols:
        X_df[i] = X_df[i].fillna(X_df.groupby('tmp')[i].transform('median'))

    # impute the binary columns
    for i in binary_cols:
        X_df[i] = X_df[i].fillna(-1)

    # impute the categorical columns
    for i in categorical_cols:
        X_df[i] = X_df[i].fillna('unknown')


    # Separate label and predictors
    y_df = X_df[target_col]
    X_df = X_df.drop(columns=[target_col])
    
    return X_df, y_df